<a href="https://colab.research.google.com/github/R0bk/ml_replications/blob/main/notes/Gibbs_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gibbs sampling

A MCMC based approach of iterative sampling conditional probability distributions to approximate the marginal probability distributions. Often used to approximate intractable marginal distributions.

A special case of Manhattan sampling

### Example

Using gaussians for easy demonstration and conditional sampling

In [ ]:
import numpy as np

In [ ]:
n_dimensions = 2

In [ ]:
mu = np.random.rand(n_dimensions)
x = np.random.rand(n_dimensions, n_dimensions)
covar = np.dot(x, x.transpose())
print(f'mu:\n{mu}')
print(f'covar:\n{covar}')

mu:
[0.69732847 0.47030449]
covar:
[[1.26934065 0.60912778]
 [0.60912778 0.58273933]]


In [ ]:
def cond_sample(x, sample_on, condition_on, mu, covar):
    '''
    Returns a random sample of one gaussian e.g. x[0] conditional on another e.g. x[1]
    '''
    x_mu = mu[sample_on] + (covar[0, 1]/covar[condition_on, condition_on]) * (x[condition_on]-mu[condition_on])
    x_sigma = (covar[sample_on, sample_on] - covar[0, 1]**2/covar[condition_on, condition_on])**0.5
    x[sample_on] = x_mu + np.random.randn()*x_sigma
    return x

In [ ]:
T = 5000
x = np.random.rand(n_dimensions)
points = np.zeros((T*2+1, n_dimensions))
for t in range(T):
    '''
    We only take cond samples in order here, for true gibbs sampling we should
    randomly select the order to take them in
    '''
    points[t*2] = x
    x = cond_sample(x, 0, 1, mu, covar)
    points[t*2+1] = x
    x = cond_sample(x, 1, 0, mu, covar)
points[-1] = x
print(points)

[[0.12707204 0.76975606]
 [1.69430932 0.76975606]
 [1.69430932 1.30712781]
 ...
 [1.58487464 0.58093051]
 [0.3714197  0.58093051]
 [0.3714197  0.45683995]]


In [ ]:
from ipywidgets import IntSlider, interact, FloatSlider
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal


@interact(t = IntSlider(min=1, max=T*2+1))
def aa_(t):
    xy_range = [*points.min(0), *points.max(0)]
    plt.figure(figsize = (15, 15))
    x, y = np.mgrid[xy_range[0]-0.5:xy_range[2]+0.5:.01, xy_range[1]-0.5:xy_range[3]+0.5:.01]
    pos = np.dstack((x, y))
    rv = multivariate_normal(mu, covar)
    plt.contourf(x, y, rv.pdf(pos))
    plt.scatter(points[:t,0], points[:t,1], s=2.5, c='#ff7f0e')
    plt.xlim(xy_range[0]-0.5, xy_range[2]+0.5)
    plt.ylim(xy_range[1]-0.5, xy_range[3]+0.5)


interactive(children=(IntSlider(value=1, description='t', max=10001, min=1), Output()), _dom_classes=('widget-…